In [4]:
import plotly.graph_objects as go

from apollo.calculations.average_directional_movement_index import (
    AverageDirectionalMovementIndexCalculator,
)
from apollo.calculations.average_true_range import AverageTrueRangeCalculator
from apollo.providers.price_data_enhancer import PriceDataEnhancer
from apollo.providers.price_data_provider import PriceDataProvider
from apollo.settings import FREQUENCY, TICKER
from apollo.utils.common import to_default_date_string

In [5]:
start_date = "1995-01-01"
end_date = "2024-09-01"

window_size = 5

price_data_provider = PriceDataProvider()
price_data_enhancer = PriceDataEnhancer()

dataframe = price_data_provider.get_price_data(
    ticker=str(TICKER),
    frequency=str(FREQUENCY),
    start_date=start_date,
    end_date=end_date,
    max_period=False,
)

dataframe = price_data_enhancer.enhance_price_data(
    price_dataframe=dataframe,
    additional_data_enhancers=["VIX", "SP500 Futures"],
)

dataframe["prev_close"] = dataframe["adj close"].shift(1)

atr_calculator = AverageTrueRangeCalculator(
    dataframe=dataframe,
    window_size=window_size,
)
atr_calculator.calculate_average_true_range()

adx_calculator = AverageDirectionalMovementIndexCalculator(
    dataframe=dataframe,
    window_size=window_size,
)
adx_calculator.calculate_average_directional_movement_index()

dataframe = dataframe.loc["2024-01-01":"2024-09-01"]

dataframe

,ticker,adj close,adj high,adj low,adj open,adj volume,close,high,low,open,...,prev_high,mdm,pdm,pdi,mdi,dx,adx,dx_adx_ampl,prev_dx,prev_dx_adx_ampl
date,,,,,,,,,,,,,,,,,,,,,
2024-01-02,SPY,468.234589,469.245079,466.094763,467.749176,1.224688e+08,472.649994,473.670013,470.489990,472.160004,...,472.573654,-0.187443,-0.186356,-0.052203,-0.052508,-0.290806,11.238824,-10.948017,12.968318,-1.152913
2024-01-03,SPY,464.410675,466.788249,463.796472,466.035339,1.026182e+08,468.790009,471.190002,468.170013,470.429993,...,469.245079,-0.609613,-0.640451,-0.171084,-0.162846,-2.466929,8.497673,-6.030744,-0.290806,-10.948017
2024-01-04,SPY,462.914764,466.560379,462.686902,463.925225,8.344532e+07,467.279999,470.959991,467.049988,468.299988,...,466.788249,-0.709604,-0.557935,-0.148014,-0.188250,-11.965668,4.405005,7.560663,-2.466929,-6.030744
2024-01-05,SPY,463.548798,466.045245,462.072696,463.122792,8.525684e+07,467.920013,470.440002,466.429993,467.489990,...,466.560379,-0.690525,-0.549375,-0.144189,-0.181236,-11.383988,1.247206,10.136782,-11.965668,7.560663
2024-01-08,SPY,470.166412,470.315005,463.925247,464.054038,7.417960e+07,474.600006,474.750000,468.299988,468.429993,...,466.045245,-0.181909,0.414452,0.094166,-0.041331,256.452450,52.288255,204.164195,-11.383988,10.136782
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2024-08-26,SPY,559.075134,562.185589,557.340465,561.457840,3.567916e+07,560.789978,563.909973,559.049988,563.179993,...,561.368158,1.773407,1.491222,0.225211,0.267828,8.643679,5.592927,3.050751,3.445044,-1.385195
2024-08-27,SPY,559.842834,560.341306,556.612752,557.779157,3.259393e+07,561.559998,562.059998,558.320007,559.489990,...,562.185589,1.273183,0.824121,0.136379,0.210692,21.411357,8.756613,12.654744,8.643679,3.050751
2024-08-28,SPY,556.592773,559.932566,553.342732,559.493909,4.094043e+07,558.299988,561.650024,555.039978,561.210022,...,560.341306,0.364542,0.577549,0.093876,0.059253,22.610008,11.527292,11.082716,21.411357,12.654744


In [6]:
value_1 = "adj close"
value_2 = "dx"
value_3 = "dx_adx_ampl"
value_4 = "pdi"

x = dataframe.index.to_numpy()

y1 = dataframe[value_1].to_numpy()
y2 = dataframe[value_2].to_numpy()
y3 = dataframe[value_3].to_numpy()
y4 = dataframe[value_4].to_numpy()

trace1 = go.Scatter(x=x, y=y1, name=value_1, yaxis="y1")
trace2 = go.Scatter(x=x, y=y2, name=value_2, yaxis="y2")
trace3 = go.Scatter(x=x, y=y3, name=value_3, yaxis="y2")
trace4 = go.Scatter(x=x, y=y4, name=value_4, yaxis="y2")

# Plot title
title = (
    f"{dataframe.iloc[0]["ticker"]} | VIX | Futures"
    f" {to_default_date_string(dataframe.index.to_numpy()[0])}"
    f" - {to_default_date_string(dataframe.index.to_numpy()[-1])}"
)

# Create the layout with two y-axes
layout = go.Layout(
    title=title,
    yaxis={},
    yaxis2={"overlaying": "y", "side": "right"},
    height=650,
)

# Create the figure and add traces to it
fig = go.Figure(
    data=[
        trace1,
        trace2,
        trace3,
        # trace4,
    ],
    layout=layout,
)

fig.update_xaxes(
    showspikes=True,
    spikemode="across",
    spikecolor="black",
    spikethickness=0.5,
)